In [ ]:
# Define n_snapshots, etc. here 
N_snapshots = 62
Increments = 20
Initial_frame = 1207

# Define inner radius, outer radius and steps 
inner_rad = 0
outer_rad = 100
step = 40

# Metal/ Gas Mass Over Time 

In [ ]:
import yt
import sys
sys.path.append("path_to/foggie/")

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
import logging, sys
logging.disable(sys.maxsize)

import matplotlib.pyplot as plt
import math
from astropy.cosmology import Planck15 as cosmo
import numpy as np
from astropy.table import Table
from mpl_toolkits.axes_grid1 import AxesGrid
from yt.utilities.cosmology import Cosmology

from foggie.utils.consistency import *
from foggie.utils.foggie_load import *
foggie_dir ="/Volumes/Student Project Drive/halo_008508/nref11c_nref9f/"
code_path = "path_to/foggie/foggie/"

track_dir = code_path + 'halo_infos/008508/nref11c_nref9f/'  # directory / -> folder
track_name = code_path + 'halo_tracks/008508/nref11n_selfshield_15/halo_track_200kpc_nref9'

In [ ]:
def get_mass_temp(n_snapshots,increments,initial_frame):
    gas_metal_mass_cgm_list = [[],[],[]]
    gas_cell_mass_cgm_list=[[],[],[]]
    for k in range(n_snapshots): 
        snap = "DD"+str(increments*k+initial_frame)
        snap_name = foggie_dir + snap + '/' + snap
        ds, refine_box = foggie_load(snap_name, track_name, halo_c_v_name=track_dir+'halo_c_v', disk_relative=False,
                             do_filter_particles=False, masses_dir=track_dir)
        
        temp_cut = [1e5,1e6]
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
         
        sphere = ds.sphere(ds.halo_center_kpc,(Rvir[0],'kpc'))
        cgm = sphere.cut_region("(obj['density'] < %.2e) & (obj['temperature'] > %.2e)" %
                                (cgm_density_max, cgm_temperature_min))

        low =cgm.cut_region(["(obj['temperature'] < {} )".format(temp_cut[0])])
        medium=cgm.cut_region(["(obj['temperature'] > {}) & (obj['temperature'] < {})"
                               .format(temp_cut[0],temp_cut[1])])
        high = cgm.cut_region(["(obj['temperature'] > {})".format(temp_cut[1])])
        data_list = [low,medium,high]
        
        
        
        
        for i in range(len(temp_cut)+1): 
            metal_mass_tot=data_list[i].quantities.total_quantity([("gas","metal_mass")])
            cell_mass_tot=data_list[i].quantities.total_quantity([("gas","cell_mass")])
        
            gas_metal_mass_cgm_list[i].append(metal_mass_tot)
            gas_cell_mass_cgm_list[i].append(cell_mass_tot)
        
    return gas_metal_mass_cgm_list,gas_cell_mass_cgm_list


In [ ]:
def get_mass_dens(n_snapshots,increments,initial_frame):
    gas_metal_mass_cgm_list = [[],[],[]]
    gas_cell_mass_cgm_list = [[],[],[]]
    for k in range(n_snapshots): 
        snap = "DD"+str(increments*k+initial_frame)
        snap_name = foggie_dir + snap + '/' + snap
        ds, refine_box = foggie_load(snap_name, track_name, halo_c_v_name=track_dir+'halo_c_v', disk_relative=False,
                             do_filter_particles=False, masses_dir=track_dir)
        
        
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
         
        sphere = ds.sphere(ds.halo_center_kpc,(Rvir[0],'kpc'))
        cgm = sphere.cut_region("(obj['density'] < %.2e) & (obj['temperature'] > %.2e)" %
                                (cgm_density_max, cgm_temperature_min))

       
        dens_cut = [1e-29,1e-27]
        low = cgm.cut_region(["(obj['density'] < {} )".format(dens_cut[0])])
        medium= cgm.cut_region(["(obj['density'] > {}) & (obj['density'] < {})".format(dens_cut[0],dens_cut[1])])
        high = cgm.cut_region(["(obj['density'] > {})".format(dens_cut[1])])
        data_list = [low,medium,high]
        
        
        for i in range(len(dens_cut)+1): 
            metal_mass_tot=data_list[i].quantities.total_quantity([("gas","metal_mass")])
            cell_mass_tot=data_list[i].quantities.total_quantity([("gas","cell_mass")])
        
            gas_metal_mass_cgm_list[i].append(metal_mass_tot)
            gas_cell_mass_cgm_list[i].append(cell_mass_tot)
        
        
    return gas_metal_mass_cgm_list, gas_cell_mass_cgm_list


In [ ]:
# Save data 
metal_cell_temp = get_mass_temp(N_snapshots,Increments,Initial_frame)
np.save('CGM_metal_cell_temp_all',metal_cell_temp)

metal_cell_dens = get_mass_dens(N_snapshots,Increments,Initial_frame)
np.save('CGM_metal_cell_dens_all',metal_cell_dens)

# Load data 
metal_cell_mass_temp_cgm = np.load('path/CGM_metal_cell_temp_all.npy',allow_pickle = True)
metal_cell_mass_dens_cgm = np.load('path/CGM_metal_cell_dens_all.npy',allow_pickle = True)

In [ ]:
conversion_factor = 1.989e33   # Convert g to M_sun 

# For Temp Cut 
metal_tot_cgm=np.divide((np.sum(metal_cell_mass_temp_cgm[0],axis=0)),conversion_factor)
cell_tot_cgm =np.divide((np.sum(metal_cell_mass_temp_cgm[1],axis=0)),conversion_factor)

metal_mass_lowtemp=np.divide(metal_mass_temp_cgm[0],conversion_factor)
metal_mass_medtemp=np.divide(metal_mass_temp_cgm[1],conversion_factor)
metal_mass_hightemp=np.divide(metal_mass_temp_cgm[2],conversion_factor)

cell_mass_lowtemp=np.divide(cell_mass_temp_cgm[0],conversion_factor)
cell_mass_medtemp=np.divide(cell_mass_temp_cgm[1],conversion_factor)
cell_mass_hightemp=np.divide(cell_mass_temp_cgm[2],conversion_factor)

In [ ]:
# For Density Cut 
metal_tot_cgm=np.divide((np.sum(metal_cell_mass_dens_cgm[0],axis=0)),conversion_factor)
cell_tot_cgm =np.divide((np.sum(metal_cell_mass_dens_cgm[1],axis=0)),conversion_factor)

metal_mass_lowdens=np.divide(metal_mass_dens_cgm[0],conversion_factor)
metal_mass_meddens=np.divide(metal_mass_dens_cgm[1],conversion_factor)
metal_mass_highdens=np.divide(metal_mass_dens_cgm[2],conversion_factor)

cell_mass_lowdens=np.divide(cell_mass_dens_cgm[0],conversion_factor)
cell_mass_meddens=np.divide(cell_mass_dens_cgm[1],conversion_factor)
cell_mass_highdens=np.divide(cell_mass_dens_cgm[2],conversion_factor)

In [ ]:
def get_list_age(n_snapshots,increments,initial_frame): 
    age_list = []
    redshift_list = []
    for k in range(n_snapshots): 
        snap = "DD"+str(increments*k+initial_frame)
        snap_name = foggie_dir + snap + '/' + snap
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
        redshift_list.append(redshift[0])
        
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age_list.append(age)
    return age_list

In [ ]:
# Get the list of age 
age_list = get_list_age(N_snapshots,Increments,Initial_frame)

# Flux 

In [ ]:
# Go to the direcotry where flux_tracking is located 
cd Desktop/FOGGIE/foggie

In [9]:
# Define n_snapshots,increments,initial_frame
n_snapshots = N_snapshots
increments = Increments
initial_frame = Initial_frame
final_frame = increments*(n_snapshots-1)+initial_frame

print("initial frame is ",initial_frame,", final frame is ",final_frame," and output_step is ",increments)
print("inner radius is ",inner_rad,", outer radius is ",outer_rad, " with step = ",step)

initial frame is  1207 , final frame is  2427  and output_step is  20
inner radius is  0 , outer radius is  100  with step =  40


In [ ]:
# If you want to get the flux of specific temp range or density range, you need to change flux_tracking.py 
# Where the CGM is defined, add extra arguments 
# Here, I made flux_tracking_low_dens.py, _medium_dens.py, _low_temp.py, _medium_temp.py
# High density and high temperature fluxes were calculated by subtracting low and med from total flux 

### Run Flux_tracking and save data

In [ ]:
# Here, you can change initial frame and final frame, output_step, "['sphere', initial_radius, final_radius,steps]" 

# All 
%run foggie/flux_tracking/flux_tracking.py --system nnishimura --halo 8508 --output DD1207-DD1227 
--output_step 20 --flux_type mass --cgm_filter  --surface "['sphere', 0, 100,40]" --kpc --save_suffix All

# Low Density 
%run foggie/flux_tracking/flux_tracking_low_dens.py --system nnishimura --halo 8508 --output DD1667-DD2427
--output_step 20 --flux_type mass --cgm_filter  --surface "['sphere', 0, 100,40]" --kpc --save_suffix lowdens

# Medium Density 
%run foggie/flux_tracking/flux_tracking_medium_dens.py --system nnishimura --halo 8508 --output DD1227-DD2427 
--output_step 40 --flux_type mass --cgm_filter  --surface "['sphere', 0, 100,40]" --kpc --save_suffix meddens

# Low Temp
%run foggie/flux_tracking/flux_tracking_low_temp.py --system nnishimura --halo 8508 --output DD1227-DD2427 
--output_step 40 --flux_type mass --cgm_filter  --surface "['sphere', 0, 100,40]" --kpc --save_suffix lowtemp

# Medium Temp 
%run foggie/flux_tracking/flux_tracking_medium_temp.py --system nnishimura --halo 8508 --output DD1227-DD2427 
--output_step 40 --flux_type mass --cgm_filter  --surface "['sphere', 0, 100,40]" --kpc --save_suffix medtemp



### Loading Data 

In [3]:
# All Data 
def getting_flux_data_all(n_snapshots,increments,initial_frame): 
    flux_data_array=[]
    for k in range(n_snapshots): 
        snapname = "DD"+str(initial_frame+increments*k)+"_fluxes_mass_All.hdf5"
        fluxdata = Table.read('path/all/'+snapname,path='all_data') 
        flux_data_array.append(fluxdata)
    return flux_data_array


# Low Density 
def getting_flux_data_lowdens(n_snapshots,increments,initial_frame): 
    flux_data_array=[]
    for k in range(n_snapshots): 
        snapname = "DD"+str(initial_frame+increments*k)+"_fluxes_mass_lowdens.hdf5"
        fluxdata = Table.read('path/lowdens/'+snapname,path='all_data') 
        flux_data_array.append(fluxdata)
    return flux_data_array

# Medium Density 
def getting_flux_data_meddens(n_snapshots,increments,initial_frame): 
    flux_data_array=[]
    for k in range(n_snapshots): 
        snapname = "DD"+str(initial_frame+increments*k)+"_fluxes_mass_meddens.hdf5"
        fluxdata = Table.read('path/meddens/'+snapname,path='all_data') 
        flux_data_array.append(fluxdata)
    return flux_data_array

# Low Temp
def getting_flux_data_lowtemp(n_snapshots,increments,initial_frame): 
    flux_data_array=[]
    for k in range(n_snapshots): 
        snapname = "DD"+str(initial_frame+increments*k)+"_fluxes_mass_lowtemp.hdf5"
        fluxdata = Table.read('path/lowtemp/'+snapname,path='all_data') 
        flux_data_array.append(fluxdata)
    return flux_data_array

# Medium Temp
def getting_flux_data_medtemp(n_snapshots,increments,initial_frame): 
    flux_data_array=[]
    for k in range(n_snapshots): 
        snapname = "DD"+str(initial_frame+increments*k)+"_fluxes_mass_medtemp.hdf5"
        fluxdata = Table.read('path/medtemp/'+snapname,path='all_data') 
        flux_data_array.append(fluxdata)
    return flux_data_array

In [10]:
all_data_array=getting_flux_data_all(n_snapshots,increments,initial_frame)
lowdens_data_array=getting_flux_data_lowdens(n_snapshots,increments,initial_frame)
meddens_data_array=getting_flux_data_meddens(n_snapshots,increments,initial_frame)
lowtemp_data_array=getting_flux_data_lowtemp(n_snapshots,increments,initial_frame)
medtemp_data_array=getting_flux_data_medtemp(n_snapshots,increments,initial_frame)

In [11]:
# High Temp

hightemp_data_net_mass_flux = []
hightemp_data_mass_flux_out = [] 
hightemp_data_mass_flux_in = []
hightemp_data_net_metal_flux = []
hightemp_data_metal_flux_in = []
hightemp_data_metal_flux_out =[]
for k in range(n_snapshots): 
    hightemp_net_mass_flux = all_data_array[k]['net_mass_flux']\
    -lowtemp_data_array[k]['net_mass_flux']-medtemp_data_array[k]['net_mass_flux']
    hightemp_data_net_mass_flux.append(hightemp_net_mass_flux) 
    hightemp_mass_flux_out = all_data_array[k]['mass_flux_out']\
    -lowtemp_data_array[k]['mass_flux_out']-medtemp_data_array[k]['mass_flux_out']
    hightemp_data_mass_flux_out.append(hightemp_mass_flux_out) 
    hightemp_mass_flux_in = all_data_array[k]['mass_flux_in']\
    -lowtemp_data_array[k]['mass_flux_in']-medtemp_data_array[k]['mass_flux_in']
    hightemp_data_mass_flux_in.append(hightemp_mass_flux_in) 
    
    hightemp_net_metal_flux = all_data_array[k]['net_metal_flux']\
    -lowtemp_data_array[k]['net_metal_flux']-medtemp_data_array[k]['net_metal_flux']
    hightemp_data_net_metal_flux.append(hightemp_net_metal_flux) 
    hightemp_metal_flux_out = all_data_array[k]['metal_flux_out']\
    -lowtemp_data_array[k]['metal_flux_out']-medtemp_data_array[k]['metal_flux_out']
    hightemp_data_metal_flux_out.append(hightemp_metal_flux_out) 
    hightemp_metal_flux_in = all_data_array[k]['metal_flux_in']\
    -lowtemp_data_array[k]['metal_flux_in']-medtemp_data_array[k]['metal_flux_in']
    hightemp_data_metal_flux_in.append(hightemp_metal_flux_in) 
    

In [18]:
# High Density 

highdens_data_net_mass_flux = []
highdens_data_mass_flux_out = [] 
highdens_data_mass_flux_in = []
highdens_data_net_metal_flux = []
highdens_data_metal_flux_in = []
highdens_data_metal_flux_out =[]
for k in range(n_snapshots): 
    highdens_net_mass_flux = all_data_array[k]['net_mass_flux']\
    -lowdens_data_array[k]['net_mass_flux']-meddens_data_array[k]['net_mass_flux']
    highdens_data_net_mass_flux.append(highdens_net_mass_flux) 
    highdens_mass_flux_out = all_data_array[k]['mass_flux_out']\
    -lowdens_data_array[k]['mass_flux_out']-meddens_data_array[k]['mass_flux_out']
    highdens_data_mass_flux_out.append(highdens_mass_flux_out) 
    highdens_mass_flux_in = all_data_array[k]['mass_flux_in']\
    -lowdens_data_array[k]['mass_flux_in']-meddens_data_array[k]['mass_flux_in']
    highdens_data_mass_flux_in.append(highdens_mass_flux_in) 
    
    highdens_net_metal_flux = all_data_array[k]['net_metal_flux']\
    -lowdens_data_array[k]['net_metal_flux']-meddens_data_array[k]['net_metal_flux']
    highdens_data_net_metal_flux.append(highdens_net_metal_flux) 
    highdens_metal_flux_out = all_data_array[k]['metal_flux_out']\
    -lowdens_data_array[k]['metal_flux_out']-meddens_data_array[k]['metal_flux_out']
    highdens_data_metal_flux_out.append(highdens_metal_flux_out) 
    highdens_metal_flux_in = all_data_array[k]['metal_flux_in']\
    -lowdens_data_array[k]['metal_flux_in']-meddens_data_array[k]['metal_flux_in']
    highdens_data_metal_flux_in.append(highdens_metal_flux_in) 
   

In [ ]:
# list of radius 
rad_list =[]
for k in range(step): 
        
    rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)

# Mass over time VS Flux (Gas and Metal)

### Gas Mass Over Time & Gas Mass Flux 

In [ ]:
def make_plot_gas(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        # get flux 
        datalist = all_data_array[k]
        net_mass_flux = datalist['net_mass_flux']
        mass_flux_out=datalist['mass_flux_out']
        mass_flux_in=datalist['mass_flux_in']
        
        net_metal_flux = datalist['net_metal_flux']
        metal_flux_out=datalist['metal_flux_out']
        metal_flux_in=datalist['metal_flux_in']
        
        f = plt.figure(figsize=(15,4))
        ax2 = f.add_subplot(121)
        ax1 = f.add_subplot(122)

        
        ax1.plot(rad_list,mass_flux_out,'r',label='Gas Outflow')
        ax1.plot(rad_list,mass_flux_in,'b',label='Gas Inflow')
        ax1.plot(rad_list,net_mass_flux,label='Net Gas Flow',color='darkviolet',linestyle='dashed')
        ax1.set_ylim([-7.5,5.0])

        ax1.axhline(y=0, color='k',linewidth=0.5)
        ax1.set_xlabel('Radius [kpc]',fontsize=14)
        ax1.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax1.legend(loc='lower left',shadow = False,fontsize ='medium')
        ax1.set_title(str(age2),fontsize=14)
         
        
        
        ax2.plot(age_list,cell_tot_cgm,label="total",color='lightcoral')
        ax2.plot(age_list,cell_mass_hightemp,label="High temp",linestyle='dashed',color='orchid')
        ax2.plot(age_list,cell_mass_medtemp,label='Medium temp',linestyle='dashed',color='mediumseagreen')
        ax2.plot(age_list,cell_mass_lowtemp,label='Low temp',linestyle='dashed',color='cornflowerblue')
        legend = ax2.legend(loc='best',shadow = False,fontsize ='medium')
        legend.get_frame().set_edgecolor('black')
        ax2.set_xlabel('age [Gyr]',fontsize=14)
        ax2.set_ylabel('Gas Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['k']
    

        for xc,c in zip(xcoords,colors):
            ax2.axvline(x=xc, label='line at x = {}'.format(xc), c=c)

### Metal Mass Over Time & Metal Mass Flux 

In [ ]:
# For All


def make_plot_metal(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        # get flux 
        datalist = all_data_array[k]
        net_mass_flux = datalist['net_mass_flux']
        mass_flux_out=datalist['mass_flux_out']
        mass_flux_in=datalist['mass_flux_in']
        
        net_metal_flux = datalist['net_metal_flux']
        metal_flux_out=datalist['metal_flux_out']
        metal_flux_in=datalist['metal_flux_in']
        
        f = plt.figure(figsize=(15,4))
        ax1 = f.add_subplot(121)
        ax2 = f.add_subplot(122)

        ax1.plot(age_list,metal_tot_cgm,label="total",color='lightcoral')
        ax1.plot(age_list,metal_mass_hightemp,label="High temp",linestyle='dashed',color='orchid')
        ax1.plot(age_list,metal_mass_medtemp,label='Medium temp',linestyle='dashed',color='mediumseagreen')
        ax1.plot(age_list,metal_mass_lowtemp,label='Low temp',linestyle='dashed',color='cornflowerblue')
        legend = ax1.legend(loc='best',shadow = False,fontsize ='medium')
        legend.get_frame().set_edgecolor('black')
        ax1.set_xlabel('age [Gyr]',fontsize=14)
        ax1.set_ylabel('Metal Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['k']
    

        for xc,c in zip(xcoords,colors):
            ax1.axvline(x=xc, label='line at x = {}'.format(xc), c=c)
 
        
        
 
        
        ax2.plot(rad_list,metal_flux_out,'r',label='Metal Outflow')
        ax2.plot(rad_list,metal_flux_in,'b',label='Metal Inflow')
        ax2.plot(rad_list,net_metal_flux,'g--',label='Net Metal Flow',color='darkviolet',linestyle='dashed')
        ax2.set_ylim([-0.02,0.05])
        ax2.axhline(y=0, color='k',linewidth=0.5)
        
        ax2.set_xlabel('Radius [kpc]',fontsize=14)
        ax2.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax2.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax2.set_title(str(age2),fontsize=14)

In [ ]:
make_plot_metal(N_snapshots,Increments,Initial_frame)
make_plot_gas(N_snapshots,Increments,Initial_frame)

# Mass over time VS 4 Flux (Gas and Metal)

### Gass Mass for Temp Cut

In [ ]:
def make_plot_gas_tempcut(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        f = plt.figure(figsize=(16,6))
        
        
        ax00 = f.add_subplot(234)
        ax0 = f.add_subplot(232)
        ax1 = f.add_subplot(233)
        ax2 = f.add_subplot(235)
        ax3 = f.add_subplot(236)

        
        
        ax00.plot(age_list,cell_tot_cgm,label="total",color='lightcoral')
        ax00.plot(age_list,cell_mass_hightemp,label="High temp",linestyle='dashed',color='orchid')
        ax00.plot(age_list,cell_mass_medtemp,label='Medium temp',linestyle='dashed',color='mediumseagreen')
        ax00.plot(age_list,cell_mass_lowtemp,label='Low temp',linestyle='dashed',color='cornflowerblue')
        legend = ax00.legend(loc='best',shadow = False,fontsize ='small')
        legend.get_frame().set_edgecolor('black')
        ax00.set_xlabel('age [Gyr]',fontsize=14)
        ax00.set_ylabel('Gas Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['k']
    

        for xc,c in zip(xcoords,colors):
            ax00.axvline(x=xc, label='line at x = {}'.format(xc), c=c)
        
        
        # All Temp 
        ax0.plot(rad_list,all_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax0.plot(rad_list,all_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax0.plot(rad_list,all_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax0.set_ylim([-7.5,5.0])

        ax0.axhline(y=0, color='k',linewidth=0.5)
        ax0.set_xlabel('Radius [kpc]',fontsize=14)
        ax0.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax0.legend(loc='lower left',shadow = False,fontsize ='small')
        ax0.set_title(str(age2),fontsize=14)
        ax0.text(88, 3, 'All', style='italic',bbox={'facecolor': 'lightcoral', 
                                                    'alpha': 0.5, 'pad': 12},fontsize=10,color='black')
 





        # Low Temp
        
        ax1.plot(rad_list,lowtemp_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax1.plot(rad_list,lowtemp_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax1.plot(rad_list,lowtemp_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax1.set_ylim([-7.5,5.0])

        ax1.axhline(y=0, color='k',linewidth=0.5)
        ax1.set_xlabel('Radius [kpc]',fontsize=14)
        ax1.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax1.legend(loc='lower left',shadow = False,fontsize ='small')
        ax1.set_title(str(age2),fontsize=14)
        ax1.text(74, 3, 'Low Temp', style='italic',bbox={'facecolor': 'lightblue', 
                                                         'alpha': 0.5, 'pad': 12},fontsize=10,color='black')


        # Medium Temp
        
        ax2.plot(rad_list,medtemp_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax2.plot(rad_list,medtemp_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax2.plot(rad_list,medtemp_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax2.set_ylim([-7.5,5.0])
        
        ax2.axhline(y=0, color='k',linewidth=0.5)
        ax2.set_xlabel('Radius [kpc]',fontsize=14)
        ax2.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax2.legend(loc='lower left',shadow = False,fontsize ='small')
        ax2.set_title(str(age2),fontsize=14)
        ax2.text(66, 3, 'Medium Temp', style='italic',bbox={'facecolor': 'lightgreen', 
                                                            'alpha': 0.5, 'pad': 12},fontsize=10,color='black')


        
        # High Temp
        
        ax3.plot(rad_list,hightemp_data_mass_flux_out[k],'r',label='Gas Outflow')
        ax3.plot(rad_list,hightemp_data_mass_flux_in[k],'b',label='Gas Inflow')
        ax3.plot(rad_list,hightemp_data_net_mass_flux[k],label='Net Gas Flow',color='darkviolet',linestyle='dashed')
        ax3.set_ylim([-7.5,5.0])
        
        ax3.axhline(y=0, color='k',linewidth=0.5)
        ax3.set_xlabel('Radius [kpc]',fontsize=14)
        ax3.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax3.legend(loc='lower left',shadow = False,fontsize ='small')
        ax3.set_title(str(age2),fontsize=14)
        ax3.text(72, 3, 'High Temp', style='italic',bbox={'facecolor': 'plum',
                                                          'alpha': 0.5, 'pad': 12},fontsize=10,color='black')



        plt.subplots_adjust(left=None, bottom=0.001, right=None, top=None, wspace=0.3, hspace=0.4)

### Gas Mass for Density Cut 

In [ ]:
def make_plot_gas_denscut(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        f = plt.figure(figsize=(16,6))
        
        
        ax00 = f.add_subplot(231)
        ax0 = f.add_subplot(232)
        ax1 = f.add_subplot(233)
        ax2 = f.add_subplot(235)
        ax3 = f.add_subplot(236)

        
        
        ax00.plot(age_list,cell_tot_cgm,label="total",color='lightcoral')
        ax00.plot(age_list,cell_mass_highdens,label="High density",linestyle='dashed',color='orchid')
        ax00.plot(age_list,cell_mass_meddens,label='Medium density',linestyle='dashed',color='mediumseagreen')
        ax00.plot(age_list,cell_mass_lowdens,label='Low density',linestyle='dashed',color='cornflowerblue')
        legend = ax00.legend(loc='best',shadow = False,fontsize ='small')
        legend.get_frame().set_edgecolor('black')
        ax00.set_xlabel('age [Gyr]',fontsize=14)
        ax00.set_ylabel('Gas Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['black']
    

        for xc,c in zip(xcoords,colors):
            ax00.axvline(x=xc, label='line at x = {}'.format(xc), c=c)
        
        
        # All Dens 
        ax0.plot(rad_list,all_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax0.plot(rad_list,all_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax0.plot(rad_list,all_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax0.set_ylim([-7.5,5.0])

        ax0.axhline(y=0, color='k',linewidth=0.5)
        ax0.set_xlabel('Radius [kpc]',fontsize=14)
        ax0.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax0.legend(loc='lower left',shadow = False,fontsize ='small')
        ax0.set_title(str(age2),fontsize=14)
        ax0.text(88, 3, 'All', style='italic',bbox={'facecolor': 'lightcoral', 
                                                    'alpha': 0.5, 'pad': 12},fontsize=10,color='black')
 





        # Low Dens
        
        ax1.plot(rad_list,lowdens_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax1.plot(rad_list,lowdens_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax1.plot(rad_list,lowdens_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax1.set_ylim([-7.5,5.0])

        ax1.axhline(y=0, color='k',linewidth=0.5)
        ax1.set_xlabel('Radius [kpc]',fontsize=14)
        ax1.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax1.legend(loc='lower left',shadow = False,fontsize ='small')
        ax1.set_title(str(age2),fontsize=14)
        ax1.text(67, 3, 'Low Density', style='italic',bbox={'facecolor': 'lightblue',
                                                            'alpha': 0.5, 'pad': 12},fontsize=10,color='black')


        # Medium Dens
        
        ax2.plot(rad_list,meddens_data_array[k]['mass_flux_out'],'r',label='Gas Outflow')
        ax2.plot(rad_list,meddens_data_array[k]['mass_flux_in'],'b',label='Gas Inflow')
        ax2.plot(rad_list,meddens_data_array[k]['net_mass_flux'],label='Net Gas Flow',
                 color='darkviolet',linestyle='dashed')
        ax2.set_ylim([-7.5,5.0])
        
        ax2.axhline(y=0, color='k',linewidth=0.5)
        ax2.set_xlabel('Radius [kpc]',fontsize=14)
        ax2.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax2.legend(loc='lower left',shadow = False,fontsize ='small')
        ax2.set_title(str(age2),fontsize=14)
        ax2.text(60, 3, 'Medium Density', style='italic',bbox={'facecolor': 'lightgreen', 
                                                               'alpha': 0.5, 'pad': 12},fontsize=10,color='black')


        
        # High Dens
        
        ax3.plot(rad_list,highdens_data_mass_flux_out[k],'r',label='Gas Outflow')
        ax3.plot(rad_list,highdens_data_mass_flux_in[k],'b',label='Gas Inflow')
        ax3.plot(rad_list,highdens_data_net_mass_flux[k],label='Net Gas Flow',color='darkviolet',linestyle='dashed')
        ax3.set_ylim([-7.5,5.0])
        
        ax3.axhline(y=0, color='k',linewidth=0.5)
        ax3.set_xlabel('Radius [kpc]',fontsize=14)
        ax3.set_ylabel('Gas Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax3.legend(loc='lower left',shadow = False,fontsize ='small')
        ax3.set_title(str(age2),fontsize=14)
        ax3.text(65, 3, 'High Density', style='italic',bbox={'facecolor': 'plum',
                                                             'alpha': 0.5, 'pad': 12},fontsize=10,color='black')



        plt.subplots_adjust(left=None, bottom=0.001, right=None, top=None, wspace=0.3, hspace=0.4)
      

### Metal Mass for Temp Cut 

In [ ]:
def make_plot_metal_tempcut(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        f = plt.figure(figsize=(16,7))
        
        
        ax00 = f.add_subplot(234)
        ax0 = f.add_subplot(232)
        ax1 = f.add_subplot(233)
        ax2 = f.add_subplot(235)
        ax3 = f.add_subplot(236)

        
        
        ax00.plot(age_list,metal_tot_cgm,'r',label="total",color='lightcoral')
        ax00.plot(age_list,metal_mass_hightemp,'m--',label="High temp",linestyle='dashed',color='orchid')
        ax00.plot(age_list,metal_mass_medtemp,'g--',label='Medium temp',linestyle='dashed',color='mediumseagreen')
        ax00.plot(age_list,metal_mass_lowtemp,'b--',label='Low temp',linestyle='dashed',color='cornflowerblue')
        legend = ax00.legend(loc='best',shadow = False,fontsize ='small')
        legend.get_frame().set_edgecolor('black')
        ax00.set_xlabel('age [Gyr]',fontsize=14)
        ax00.set_ylabel('Metal Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['k']
    

        for xc,c in zip(xcoords,colors):
            ax00.axvline(x=xc, label='line at x = {}'.format(xc), c=c)
        
        
       # All Temp
        
        ax0.plot(rad_list,all_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax0.plot(rad_list,all_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax0.plot(rad_list,all_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax0.set_ylim([-0.02,0.05])

        ax0.axhline(y=0, color='k',linewidth=0.5)
        ax0.set_xlabel('Radius [kpc]',fontsize=14)
        ax0.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax0.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax0.set_title(str(age2),fontsize=14)
        ax0.text(4, 0.04, 'All', style='italic',bbox={'facecolor': 'lightcoral',
                                                      'alpha': 0.5, 'pad': 12},fontsize=12,color='black')




        # Low Temp
        
        ax1.plot(rad_list,lowtemp_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax1.plot(rad_list,lowtemp_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax1.plot(rad_list,lowtemp_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax1.set_ylim([-0.02,0.05])

        ax1.axhline(y=0, color='k',linewidth=0.5)
        ax1.set_xlabel('Radius [kpc]',fontsize=14)
        ax1.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax1.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax1.set_title(str(age2),fontsize=14)
        ax1.text(4, 0.04, 'Low Temp', style='italic',bbox={'facecolor': 'lightblue', 
                                                           'alpha': 0.5, 'pad': 12},fontsize=12,color='black')


        # Medium Temp
        
        ax2.plot(rad_list,medtemp_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax2.plot(rad_list,medtemp_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax2.plot(rad_list,medtemp_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax2.set_ylim([-0.02,0.05])
        
        ax2.axhline(y=0, color='k',linewidth=0.5)
        ax2.set_xlabel('Radius [kpc]',fontsize=14)
        ax2.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax2.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax2.set_title(str(age2),fontsize=14)
        ax2.text(4, 0.04, 'Medium Temp', style='italic',bbox={'facecolor': 'lightgreen',
                                                              'alpha': 0.5, 'pad': 12},fontsize=12,color='black')


        
        # High Temp
        
        ax3.plot(rad_list,hightemp_data_metal_flux_out[k],'r',label='Metal Outflow')
        ax3.plot(rad_list,hightemp_data_metal_flux_in[k],'b',label='Metal Inflow')
        ax3.plot(rad_list,hightemp_data_net_metal_flux[k],label='Net Metal Flow',color='darkviolet',linestyle='dashed')
        ax3.set_ylim([-0.02,0.05])
        
        ax3.axhline(y=0, color='k',linewidth=0.5)
        ax3.set_xlabel('Radius [kpc]',fontsize=14)
        ax3.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax3.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax3.set_title(str(age2),fontsize=14)
        ax3.text(4, 0.04, 'High Temp', style='italic',bbox={'facecolor': 'plum', 
                                                            'alpha': 0.5, 'pad': 12},fontsize=12,color='black')



        plt.subplots_adjust(left=None, bottom=0.001, right=None, top=None, wspace=0.3, hspace=0.4)

### Metal Mass for Density Cut 

In [ ]:
def make_plot_metal_denscut(n_snapshots,increments,initial_frame):
    rad_list =[]
    for k in range(step): 
        
        rad_list.append(inner_rad+(outer_rad-inner_rad)/(step)*k)
    
    for k in range(n_snapshots): 
        
        snapname = "DD"+str(initial_frame+increments*k)
        snap = "DD"+str(initial_frame+increments*k)
        rvir_masses = Table.read('path_to/rvir_masses.hdf5',path ='all_data')
        Rvir = rvir_masses['radius'][rvir_masses['snapshot']==snap]
        redshift =rvir_masses['redshift'][rvir_masses['snapshot']==snap]
    
        co = Cosmology()
        age = co.t_from_z(redshift[0]).in_units("Gyr")
        age2="{:.2f}".format(age)
        
        f = plt.figure(figsize=(16,7))
        
        
        ax00 = f.add_subplot(231)
        ax0 = f.add_subplot(232)
        ax1 = f.add_subplot(233)
        ax2 = f.add_subplot(235)
        ax3 = f.add_subplot(236)

        
        
        ax00.plot(age_list,metal_tot_cgm,'r',label="total",color='lightcoral')
        ax00.plot(age_list,metal_mass_highdens,'m--',label="High density",linestyle='dashed',color='orchid')
        ax00.plot(age_list,metal_mass_meddens,'g--',label='Medium density',linestyle='dashed',color='mediumseagreen')
        ax00.plot(age_list,metal_mass_lowdens,'b--',label='Low density',linestyle='dashed',color='cornflowerblue')
        legend = ax00.legend(loc='best',shadow = False,fontsize ='small')
        legend.get_frame().set_edgecolor('black')
        ax00.set_xlabel('age [Gyr]',fontsize=14)
        ax00.set_ylabel('Metal Mass [$M_{\odot}$]',fontsize=14)
        xcoords = [age_list[k]]
        # colors for the lines
        colors = ['k']
    

        for xc,c in zip(xcoords,colors):
            ax00.axvline(x=xc, label='line at x = {}'.format(xc), c=c)
        
        
       # All Dens
        
        ax0.plot(rad_list,all_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax0.plot(rad_list,all_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax0.plot(rad_list,all_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax0.set_ylim([-0.02,0.05])

        ax0.axhline(y=0, color='k',linewidth=0.5)
        ax0.set_xlabel('Radius [kpc]',fontsize=14)
        ax0.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax0.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax0.set_title(str(age2),fontsize=14)
        ax0.text(4, 0.04, 'All', style='italic',bbox={'facecolor': 'lightcoral', 
                                                      'alpha': 0.5, 'pad': 12},fontsize=12,color='black')




        # Low Dens
        
        ax1.plot(rad_list,lowdens_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax1.plot(rad_list,lowdens_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax1.plot(rad_list,lowdens_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax1.set_ylim([-0.02,0.05])

        ax1.axhline(y=0, color='k',linewidth=0.5)
        ax1.set_xlabel('Radius [kpc]',fontsize=14)
        ax1.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax1.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax1.set_title(str(age2),fontsize=14)
        ax1.text(4, 0.04, 'Low Density', style='italic',bbox={'facecolor': 'lightblue', 
                                                              'alpha': 0.5, 'pad': 12},fontsize=12,color='black')


        # Medium Dens
        
        ax2.plot(rad_list,meddens_data_array[k]['metal_flux_out'],'r',label='Metal Outflow')
        ax2.plot(rad_list,meddens_data_array[k]['metal_flux_in'],'b',label='Metal Inflow')
        ax2.plot(rad_list,meddens_data_array[k]['net_metal_flux'],label='Net Metal Flow',
                 color='darkviolet',linestyle='dashed')
        ax2.set_ylim([-0.02,0.05])
        
        ax2.axhline(y=0, color='k',linewidth=0.5)
        ax2.set_xlabel('Radius [kpc]',fontsize=14)
        ax2.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax2.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax2.set_title(str(age2),fontsize=14)
        ax2.text(4, 0.04, 'Medium Density', style='italic',bbox={'facecolor': 'lightgreen', 
                                                                 'alpha': 0.5, 'pad': 12},fontsize=12,color='black')


        
        # High Dens
        
        ax3.plot(rad_list,highdens_data_metal_flux_out[k],'r',label='Metal Outflow')
        ax3.plot(rad_list,highdens_data_metal_flux_in[k],'b',label='Metal Inflow')
        ax3.plot(rad_list,highdens_data_net_metal_flux[k],label='Net Metal Flow',color='darkviolet',linestyle='dashed')
        ax3.set_ylim([-0.02,0.05])
        
        ax3.axhline(y=0, color='k',linewidth=0.5)
        ax3.set_xlabel('Radius [kpc]',fontsize=14)
        ax3.set_ylabel('Metal Mass Flux [$M_{\odot}$/yr]',fontsize=14)
        legend = ax3.legend(loc='upper right',shadow = False,fontsize ='medium')
        ax3.set_title(str(age2),fontsize=14)
        ax3.text(4, 0.04, 'High Density', style='italic',bbox={'facecolor': 'plum',
                                                               'alpha': 0.5, 'pad': 12},fontsize=12,color='black')



        plt.subplots_adjust(left=None, bottom=0.001, right=None, top=None, wspace=0.3, hspace=0.4)
       

In [ ]:
make_plot_gas_tempcut(N_snapshots,Increments,Initial_frame)
make_plot_gas_denscut(N_snapshots,Increments,Initial_frame)
make_plot_metal_tempcut(N_snapshots,Increments,Initial_frame)
make_plot_metal_denscut(N_snapshots,Increments,Initial_frame)